In [409]:
import os
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from langchain_community.llms import Ollama

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_excel('/Users/ahmet/Desktop/Study/second_semester/nlp/project/data/CodingDiscourseAnalysis/dataset.xlsm')
# Dropping topic because it has only one value. It might be usefull if we had more data
df = df.drop(columns=['Old Code Book', 'Memo', 'Chat0CREW1B', 'Chat0CREW1', 'Response Number', 'Message Time', 'CollapsR2DiscussionTypeInterpNothers', 'R2DiscussionTypeInterpNothers'])
df.columns = [''.join([word[0].upper() + word[1:] for word in col.split()]) for col in df.columns]
df.columns = [col.replace(' ', '') for col in df.columns]

In [8]:
# Identify the columns that start with R1 and R2 and have the same suffix
r1_columns = [col for col in df.columns if col.startswith('R1') or col.startswith('BinaryR1')]
r2_columns = [col for col in df.columns if col.startswith('R2') or col.startswith('BinaryR2')]

# Make sure that the suffixes match
matched_columns = [(r1, r2) for r1 in r1_columns for r2 in r2_columns if r1[2:] == r2[2:] or (r1[9:] == r2[9:] and r1.startswith('Binary') and r2.startswith('Binary'))]
matched_columns
#
## Iterate over these pairs of columns
for r1_col, r2_col in matched_columns:
    # Find rows where the R2 version has NaN and the R1 version has a value
    condition = df[r2_col].isna() & df[r1_col].notna()
    if condition.any():# and "Dialogic" not in r1_col:
        print(f'Column {r2_col} has NaN where {r1_col} has a value')
        print(df.loc[condition, [r1_col, r2_col, 'BinaryR2Uptake']].head())
        print(df.loc[condition, ['Message']].head())

Column R2DialogicSpell has NaN where R1DialogicSpell has a value
    R1DialogicSpell  R2DialogicSpell  BinaryR2Uptake
240                              NaN               0
261           Begin              NaN               0
268                              NaN               0
273                              NaN               1
274                              NaN               0
                                               Message
240  she was smiling by the end and the book told m...
261  As for the second question I believe that the ...
268               Hey guys I got through the next page
273                            I know live without him
274       im just back at the screen we were at before
Column BinaryR2DialogicSpell has NaN where BinaryR1DialogicSpell has a value
     BinaryR1DialogicSpell  BinaryR2DialogicSpell  BinaryR2Uptake
315                      1                    NaN               0
316                      1                    NaN               0
            

In [9]:
df.drop(columns=r1_columns, inplace=True)

In [10]:
dot1_columns = [col for col in df.columns if col.endswith('.1')]
original_columns = [col.rstrip('.1') for col in dot1_columns if col.rstrip('.1') in df.columns]

# For each pair of columns, if one column has a value and the other does not, keep the value
for orig_col, dot1_col in zip(original_columns, dot1_columns):
    if (df[orig_col].isna() & df[dot1_col].notna()).any():
        print(f'Column {orig_col} has a value where {dot1_col} has NaN')

# Remove the .1 columns
df = df.drop(columns=dot1_columns)

In [289]:
df.head()

,Course,BookID,Topic,Bookclub,Pseudonym,Message,IsAnswer,Page,R2DiscussionType,R2DialogicSpell,BinaryR2DialogicSpell,R2Uptake,BinaryR2Uptake,R2Question,R2Pivot
0,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,pim-01,Hello.,No,10,Social,1.0,1.0,NaN,0,NaN,NaN
1,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,pim-01,My assumption is,No,10,Seminar,1.0,1.0,NaN,0,NaN,Social/Procedure/UX to Seminar
2,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,pim-01,that the emphasis on barbarism implies that sh...,No,10,Seminar,1.0,1.0,NaN,0,NaN,NaN
3,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,pim-03,I agree with Cassandra's noticing,No,10,Seminar,1.0,1.0,Affirm,1,NaN,NaN
4,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,pim-03,of the author's word choice of barbarism.,No,10,Seminar,1.0,1.0,Affirm,1,NaN,NaN


In [38]:
uptakes = df['R2Uptake'].value_counts(dropna=False).index.to_list()
uptakes[0] = 'NaN'

In [217]:
examples = []
for uptake in uptakes:
    print(uptake)
    if uptake == 'NaN':
        print(df.loc[df['R2Uptake'].isna(), 'Message'][:5].head().values)
        examples.append({uptake : df.loc[df['R2Uptake'].isna(), 'Message'][:5].head().values})
    else:
        print(df.loc[df['R2Uptake'] == uptake, 'Message'][:5].head().values)
        examples.append({uptake: df.loc[df['R2Uptake'] == uptake, 'Message'][:5].head().values})

NaN
['Hello. ' 'My assumption is'
 'that the emphasis on barbarism implies that she sent him to the lion.'
 'I loved the addition of '
 'Her lover would die and never love another." ']
Affirm
["I agree with Cassandra's noticing "
 "of the author's word choice of barbarism." 'I agree with Ashely '
 'I also agree with Ashley' ' I like this idea!']
Elaborate
[" that the Princess's barbaric traits will lead the princess to send her lover to his death."
 "that the king's adoration of his daughter would stop him from sending his daughter into the arena."
 'I tried looking for a "particpants" button or something and I couldn\'t find one.'
 "I sent an email to the coordinator and I hope to hear back from them, but I wouldn't mind setting up a discussion in our class page too"
 "I'm going to create a discussion called Book Club 4 and see if we can see if there is anyone else who is part of that group."]
Filler
["I had not considered Ashley's idea that this experience might be an "
 'opportun'
 

In [280]:
def agent(examples, line):
    
    # os.environ['SERPER_API_KEY'] = 'YOUR API KEY'
    llm =  Ollama(model = 'openhermes')
    # search_tool = SerperDevTool()
    
    true = ''
    for exp in examples:
        for key, value in exp.items():
            for v in value:
                true += f'\n\n{v} // {key}'
                
    researcher = Agent(llm = llm,
                    role = "Discourse Analyst",
                    goal = """To analyze the sentece and categorize the uptake of the messages.
                    """,
                    backstory = f"""You are an experienced discourse analyst. You are given a sentence and you are expected to categorize
                    the uptake of the sentence one of the following categories: Nan, Affirm, Elaborate, Filler, Clarify, Disagree.
                    Examples:{true}
                    """,
                    allow_delegation = False,
                    # tools = [search_tool]
                    )

    
    task1 =  Task(description = f'Provide me uptake category of this line \n\n{line}',
            expected_output = """The uptake category of the sentence in the following format:
            
            sentence // uptake_category
            """,
            agent = researcher
                )
    


    crew = Crew(agents=[researcher], tasks=[task1], verbose=2)

    task_output = crew.kickoff(inputs = {'topic': 'discourse analysis'})
    
    return task_output

In [281]:
test_line = df[['Message', 'R2Uptake']].iloc[np.random.randint(0, len(df), 1)]
display(test_line)
result = agent(examples, test_line['Message'].values[0])

,Message,R2Uptake
69,"Without someone seeing her give the signal, th...",NaN


2024-05-02 15:59:57,084 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

Without someone seeing her give the signal, they really cannot accuse her of helping him
 [DEBUG]: == [Discourse Analyst] Task output: "Without someone seeing her give the signal, they really cannot accuse her of helping him" // NaN




In [285]:
test_line = df[['Message', 'R2Uptake']].iloc[np.random.randint(0, len(df), 1)]
display(test_line)
result = agent(examples, test_line['Message'].values[0])

,Message,R2Uptake
519,"it says Alexandrea, Matthew and Sara looked at...",Clarify


2024-05-02 16:01:19,553 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

it says Alexandrea, Matthew and Sara looked at this 18 days ago. How is this possible? I thought the activity just opened up this week?
 [DEBUG]: == [Discourse Analyst] Task output: it says Alexandrea, Matthew and Sara looked at this 18 days ago. How is this possible? I thought the activity just opened up this week? // Clarify




In [287]:
test_line = df[['Message', 'R2Uptake']].iloc[np.random.randint(0, len(df), 1)]
display(test_line)
result = agent(examples, test_line['Message'].values[0])

,Message,R2Uptake
337,"Ok, that's how I understand it as well. Let's ...",Affirm


2024-05-02 16:01:56,031 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

Ok, that's how I understand it as well. Let's wait a few more minutes to give Julie a chance to join us. Sound good?
 [DEBUG]: == [Discourse Analyst] Task output: Ok, that's how I understand it as well. Let's wait a few more minutes to give Julie a chance to join us. // Affirm




In [288]:
test_line = df[['Message', 'R2Uptake']].iloc[np.random.randint(0, len(df), 1)]
display(test_line)
result = agent(examples, test_line['Message'].values[0])

,Message,R2Uptake
302,"Perfect. ""See"" you then.",Affirm


2024-05-02 16:03:13,291 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

Perfect. "See" you then.
 [DEBUG]: == [Discourse Analyst] Task output: "Perfect. 'See' you then." // NaN




In [292]:
df_train = df.copy()

In [297]:
for exp in examples:
    for k, v in exp.items():
        for mes in v:
            df_train.drop(df_train[df_train['Message'] == mes].index, inplace=True)

In [304]:
results = []
for message in df_train['Message']:
    agent_response = agent(examples, message)
    results.append({message:agent_response})

2024-05-02 16:08:02,839 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

 Sadly  the barbaric nature of the king and princess is still alive today in crimes of passion."


2024-05-02 16:08:15,466 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == [Discourse Analyst] Task output: Sadly the barbaric nature of the king and princess is still alive today in crimes of passion." // Elaborate


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

Submitted


2024-05-02 16:08:17,331 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


...

 [DEBUG]: == [Discourse Analyst] Task output: The sentence "I have no guess as to which is behind the door. Is it unclear whether or not she is even instructing him to choose the door on the right side. She could have raised her right hand to signal “DON’T go in that door, the tiger is there.”" // Clarify


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

Part 2: The daughter is “the apple of the king’s eye”, therefore, I believe he will not disclose this knowledge to anyone. This would save her from a trial.


2024-05-02 16:45:40,447 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == [Discourse Analyst] Task output: Part 2: The daughter is “the apple of the king’s eye”, therefore, I believe he will not disclose this knowledge to anyone. This would save her from a trial. // Elaborate


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

Part 1: Honestly hard to say, though I'd like to believe she spared the one she loved. That said, If she is as barbaric as her father, she could very well have sent him to his death. If she can't have him, no one can.


2024-05-02 16:45:45,368 - 8653306560 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == [Discourse Analyst] Task output: Part 1: Honestly hard to say, though I'd like to believe she spared the one she loved. That said, If she is as barbaric as her father, she could very well have sent him to his death. If she can't have him, no one can. // NaN


 [DEBUG]: == Working Agent: Discourse Analyst
 [INFO]: == Starting Task: Provide me uptake category of this line 

Part 2: The king most certainly spares his daughter, one way or another. She is his greatest pride. I say he puts on a show for the people to act as though the law is being upheld, but behind the doors are suiters of his choosing. He keeps his daughter safe and she has to marry someone he believes is the best choice for her.
 [DEBUG]: == [Discourse Analyst] Task output: Part 2: The king most certainly spares his daughter, one way or another. She is his greatest pride. I say he puts on a show for the people to act as though the law is being upheld, but behind the doors are suiters of his choosing. He keeps

In [410]:
df_format = []
for r in results:
    for k, v in r.items():
        df_format.append({'Message': k, 'Response': v})

In [411]:
df_results = pd.DataFrame(df_format)

In [412]:
df_results['Generated_labels'] = 'Not Found'

for row in df_results.iterrows():
    try:
        df_results['Generated_labels'].iloc[row[0]] = row[1][1].split('//')[1].strip()
    except:
        for uptake in uptakes:
            if uptake in row[1][1]:
                df_results['Generated_labels'].iloc[row[0]] = uptake
            else:
                pass
        

In [413]:
df_cleaned = df_results[df_results['Generated_labels'].isin(uptakes)]
df_last = pd.merge(df_cleaned, df_train[['Message', 'R2Uptake']], on='Message', how='inner')

In [414]:
print('Accuracy (uptake): {:.2f}%'.format(sum(df_last['Generated_labels'] == df_last['R2Uptake']) / len(df_last['Generated_labels']) * 100))

Accuracy (uptake): 27.04%


In [415]:
df_notnan = df_last[~df_last['R2Uptake'].isna()]
df_nan = df_last[df_last['R2Uptake'].isna()]

In [416]:
print('Accuracy (uptake) for not nan labels: {:.2f}%'.format(sum(df_notnan['Generated_labels'] == df_notnan['R2Uptake']) / len(df_notnan['Generated_labels']) * 100))
print('Accuracy (uptake) for NaN labels: {:.2f}%'.format(sum(df_nan['Generated_labels'] == df_nan['R2Uptake']) / len(df_nan['Generated_labels']) * 100))

Accuracy (uptake) for not nan labels: 45.99%
Accuracy (uptake) for NaN labels: 0.00%
